In [1]:
from om_api_client import OmApiClient, timeseriesMetadataToDataFrame, featuresToDataFrame, featuresToGeoJSON
import pandas
import plotly.express as px
import logging
from datetime import datetime

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

parameters

In [2]:
begin_date = "1990-01-01"
end_date = str(datetime.now())[0:10]
feature_id = "FAAC49BA633EFF325BE5D2BA81BE14574A268ABA"
observation_identifiers_csv = "data/ina_timeseries.csv"
observed_property = "Discharge"
aggregation_duration = "P1M"

Instantiate client

In [3]:
client = OmApiClient()

retrieve timeseries using site and variable ids

In [ ]:

ts_metadata = client.getTimeseries(
    feature=feature_id,
    observedProperty=observed_property,
    aggregationDuration=aggregation_duration)

Inspect metadata

In [5]:
len(ts_metadata["member"])

3

Select observation identifier

In [6]:
observationIdentifier = ts_metadata["member"][0]["id"]

In [7]:
observationIdentifier

'18EB307E3D1C45D3A2842D710A41001AB5083041'

retrieve data

In [ ]:

data = client.getData(
    begin_date, 
    end_date,
    observationIdentifier = observationIdentifier)

convert to dataframe and plot

In [9]:
df = pandas.DataFrame(data)
df["date"] = pandas.to_datetime(df["date"])
df = df.set_index("date")
px.line(df.reset_index(), x="date", y="value")

read observation identifiers from csv, retrieve data and plot together

In [12]:
from io import StringIO
csv_data = """stream,station_name,station_id,variable,ObservationId
URUGUAY,Paso de los Libres,72,monthly discharge,8272678FE72DB91CD511E653099DB3219DEE615B
URUGUAY,Santo Tomé,68,monthly discharge,18A95E501B2C4EEC191BE2215D87DDF107AF8A42
URUGUAY,San Javier,65,monthly discharge,9DB2FD5D21BE8FFDF36B699E3CC607CD98FFFB03
PARANA,Paraná,29,monthly discharge,B569750A1B728AB62D03460068CF80CCCD011D13
PARANA,Santa Fe,30,monthly discharge,97A1C9210A637D94FB29B5BACB0500E0F353AB04
PARANA,Barranqueras,20,monthly discharge,24F40961A057CE7DC723EE86BBA3B39729F03CBC
"""
stations = pandas.read_csv(StringIO(csv_data))


In [ ]:
df_batch = client.getDataBatch(
        begin_date,
        end_date,
        observationIdentifiers = stations,
        recursive = False
)

px.line(df_batch, x = "date", y = "value", color = "ObservationId")

get metadata first page

In [ ]:
ts_metadata = client.getTimeseries(
    observedProperty=observed_property,
    limit=50)
ts_metadata.keys()

In [17]:
len(ts_metadata["member"]) # 50

50

In [18]:
ts_metadata["completed"] # False

False

get metadata all pages

In [ ]:
ts_metadata = client.getTimeseriesWithPagination(
    observedProperty=observed_property,
    limit=50)
len(ts_metadata["member"]) #  > 50

In [20]:
ts_metadata["member"][0]

{'parameter': [{'name': 'source',
   'value': 'Argentina, Instituto Nacional del Agua (INA)'},
  {'name': 'sourceId', 'value': 'argentina-ina'},
  {'name': 'observedPropertyDefinition',
   'value': 'Discharge Data type: Continuous Value type: Derived Value Units: metros cúbicos por segundo Units type: Flow Unit abbreviation:  No data value: -9999 Speciation:'},
  {'name': 'originalObservedProperty', 'value': 'Discharge'}],
 'observedProperty': {'href': 'http://hydro.geodab.eu/hydro-ontology/concept/78',
  'title': 'Discharge, stream'},
 'phenomenonTime': {'end': '2020-08-31T03:00:00Z',
  'begin': '2020-08-06T03:00:00Z'},
 'featureOfInterest': {'href': '00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C'},
 'id': '04CE047C0DF5A150C3FD9F7FD9B75F7E6098EFC9',
 'type': 'TimeSeriesObservation',
 'result': {'metadata': {},
  'defaultPointMetadata': {'uom': 'metros cúbicos por segundo',
   'interpolationType': {'href': 'http://www.opengis.net/def/waterml/2.0/interpolationType/Continuous',
    'title': '

In [21]:
df_ts = timeseriesMetadataToDataFrame(ts_metadata)
df_ts.head(5)

,sourceId,observedProperty,beginDate,endDate,featureId,ObservationId,uom,interpolationType,aggregationDuration
0,argentina-ina,"Discharge, stream",2020-08-06T03:00:00Z,2020-08-31T03:00:00Z,00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C,04CE047C0DF5A150C3FD9F7FD9B75F7E6098EFC9,metros cúbicos por segundo,Continuous/Instantaneous,None
1,argentina-ina,"Discharge, stream",2020-08-01T03:00:00Z,2020-08-01T03:00:00Z,00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C,33DC6C9A6EC6777C8C1E57391C0539749DC40BE3,metros cúbicos por segundo,Average in succeeding interval,P1M
2,argentina-ina,"Discharge, stream",2020-08-01T03:00:00Z,2020-08-01T03:00:00Z,00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C,363201EC2149C679D5A7A65C126916A13FABDD5D,metros cúbicos por segundo,Average in succeeding interval,P1M
3,argentina-ina,"Discharge, stream",2020-08-06T03:00:00Z,2020-08-31T03:00:00Z,00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C,68DA7DC5CB959B851B4461E8CA6A875790B8F919,metros cúbicos por segundo,Average in succeeding interval,P1D
4,argentina-ina,"Discharge, stream",2020-08-01T03:00:00Z,2020-08-01T03:00:00Z,00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C,D170EC5D1100F30D913E60AE223E44DDDCCC7078,metros cúbicos por segundo,Average in succeeding interval,P1M


get features first page

In [ ]:
features = client.getFeatures(
    observedProperty=observed_property,
    limit=50)
features.keys()

In [23]:
len(features["results"])

50

get features with pagination

In [ ]:
features = client.getFeaturesWithPagination(
    observedProperty=observed_property,
    limit=50)
len(features["results"])

In [25]:
features["results"][0]

{'shape': {'coordinates': [-57.938011, -31.273969], 'type': 'Point'},
 'parameter': [{'name': 'country', 'value': 'Argentina'},
  {'name': 'source', 'value': 'Argentina, Instituto Nacional del Agua (INA)'},
  {'name': 'sourceId', 'value': 'argentina-ina'},
  {'name': 'identifier', 'value': 'argentina-ina:alturas_prefe:89'}],
 'name': 'Aporte Salto Grande',
 'id': '00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C',
 'relatedParty': []}

In [26]:
featuresToDataFrame(features)




,longitude,latitude,country,sourceId,identifier,name,id,author
0,-57.938011,-31.273969,Argentina,argentina-ina,argentina-ina:alturas_prefe:89,Aporte Salto Grande,00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C,None
1,-60.780556,-31.491222,Argentina,argentina-ina,argentina-ina:alturas_bdhi:103,Recreo - Ruta Provincial nº 70,02E07E2F72A815E237B7896D79BC68193EF3A0A6,None
2,-58.558333,-28.995000,Argentina,argentina-ina,argentina-ina:sat2:2832,Corriente - Paso Lucero,083988DC8C2E39D6E0C82B296A857F6A088B8028,None
3,-57.633333,-30.250000,Argentina,argentina-ina,argentina-ina:alturas_prefe:74,Monte Caseros,0DF4C4284AA14E0A545C0F855B4F6FF558693CEE,None
4,-55.883333,-27.366667,Argentina,argentina-ina,argentina-ina:alturas_prefe:14,Posadas,23137FB463C0902450418E9E2F0D38C719419AA9,None
5,-54.756333,-26.750750,Argentina,argentina-ina,argentina-ina:alturas_bdhi:2038,Paranay - El Alcazar,255B2448BEBEAF022A54211BB36BAE67020214E8,None
6,-59.076806,-32.308500,Argentina,argentina-ina,argentina-ina:alturas_bdhi:2,Rosario del Tala,2F12488193F66939384E07C2FD757FDAF2781D52,None
7,-59.178333,-34.589444,Argentina,argentina-ina,argentina-ina:sat2:2128,Lujan - Lujan,329BB00817E03FB6C1E86BF7625BA824692C9A95,None
8,-56.727244,-27.482557,Argentina,argentina-ina,argentina-ina:alturas_prefe:87,Yacyretá afluente,329E5E2914B19DB23520D64DFA541CC83DE345F2,None
9,-59.217222,-29.757222,Argentina,argentina-ina,argentina-ina:sat2:2199,Corrientes - Los Laureles,33D8EF13796FA55AD42D54A54EF43D6DEB4CB529,None


In [27]:
featuresToGeoJSON(features)

{'type': 'FeatureCollection',
 'features': [{'geometry': {'coordinates': [-57.938011, -31.273969],
    'type': 'Point'},
   'properties': {'longitude': -57.938011,
    'latitude': -31.273969,
    'country': 'Argentina',
    'sourceId': 'argentina-ina',
    'identifier': 'argentina-ina:alturas_prefe:89',
    'name': 'Aporte Salto Grande',
    'id': '00D3297FBC6A70359AC9C78EC0A56AE2EBBD8B6C',
    'author': None}},
  {'geometry': {'coordinates': [-60.7805555555556, -31.4912222222222],
    'type': 'Point'},
   'properties': {'longitude': -60.7805555555556,
    'latitude': -31.4912222222222,
    'country': 'Argentina',
    'sourceId': 'argentina-ina',
    'identifier': 'argentina-ina:alturas_bdhi:103',
    'name': 'Recreo - Ruta Provincial nº 70',
    'id': '02E07E2F72A815E237B7896D79BC68193EF3A0A6',
    'author': None}},
  {'geometry': {'coordinates': [-58.5583333333333, -28.995], 'type': 'Point'},
   'properties': {'longitude': -58.5583333333333,
    'latitude': -28.995,
    'country': 'A

In [ ]:
px.scatter(featuresToDataFrame(features), "longitude", "latitude", hover_name = "name", title = "features")